

  `####################################################################################`

`####` <l style= "color:yellow; font-size: 20px; ">&emsp;Hack-a-thon GeeksforGeeks Hiring Data Science &emsp;</li> 

 `####`<l style="font-family: 'Courier New', monospace; color: #E6E6FA; font-size: 20px; ">&emsp;Submitted by Mortuza Reza &emsp; &emsp; </l> 

 `####`<l style="font-family: 'Courier New', monospace; color: orange; font-size: 16px; font-style:italic; ">&emsp;m.reza@usask.ca &emsp; &emsp; </l>

 `####`<l style="font-family: 'Courier New', monospace; color: orange; font-size: 16px; ">&emsp;iamreza.ca &emsp; &emsp; </l>

  `####################################################################################`

# Problem 1

## Data preparation

I converted the JSON dataset into a relational database format to facilitate detailed analysis. Utilizing Django, I stored the data in a PostgreSQL database. The following code snippets outline the process of constructing the database with various tables.

In [60]:
import json

In [64]:
with open('data.json') as f:
    data = json.load(f)

In [ ]:
# List of Resstaurants where no sublocations are present

location_without_sublocations = []
restaurants_lws = 0
for sub in data:
    try:
        location_without_sublocations.append(sub)
        res = data[sub]['restaurants']
        restaurants_lws +=1              
        mainloc = MainLocation(name = sub)
        # subloc= SubLocation(name = sub)
        mainloc.save()
        # subloc.save()
        loc = Location(main = mainloc)
        loc.save()               
        for rest in data[sub]['restaurants']:
            try:
                restaurant = Restaurant(
                    rid = rest,
                    name = data[sub]['restaurants'][rest]['name'],
                    rating = data[sub]['restaurants'][rest]['rating'],
                    rating_count = data[sub]['restaurants'][rest]['rating_count'],
                    cost = data[sub]['restaurants'][rest]['cost'],
                    address = data[sub]['restaurants'][rest]['address'],
                    cuisine = data[sub]['restaurants'][rest]['cuisine'],
                    lic_no = data[sub]['restaurants'][rest]['lic_no'],
                    location = loc,
                    link =  data[sub]['restaurants'][rest]['link']
                )
                restaurant.save()
            except:
                try:
                    new = data[sub]['restaurants'][rest]['price']
                    restaurant = Restaurant(
                        rid = rest,
                        name = data[sub]['restaurants'][rest]['name'],
                        rating = data[sub]['restaurants'][rest]['rating'],
                        cost = data[sub]['restaurants'][rest]['price'],
                        cuisine = data[sub]['restaurants'][rest]['type'],
                        link =  data[sub]['restaurants'][rest]['link'],
                        location = loc,
                    )
                    restaurant.save()
                except:
                    print(data[sub]['restaurants'][rest]['price'])
                    pass
    except:
        pass
        # print(data[sub].keys())

In [ ]:

# List of Resstaurants in cities where sublocations are present

sublocations = []
location_with_sublocations = []
restaurants_in_sublocations = 0
for sub in data:
    try:
        res = data[sub]['restaurants']
        # location_with_sublocations.append(sub)
        # restaurants_in_sublocations +=1
    except:
        for s in data[sub]:
            if s != 'link':
                # print(s)
                try:
                    sublocations.append(s)
                    res = data[sub][s]['restaurants']
                    restaurants_in_sublocations +=1                 
                    mainloc = MainLocation(name = sub)
                    subloc= SubLocation(name = s)
                    mainloc.save()
                    subloc.save()
                    loc = Location(main = mainloc, sub = subloc)
                    loc.save()               
                    for rest in data[sub][s]['restaurants']:
                        try:
                            restaurant = Restaurant(
                                        rid = rest,
                                        name = data[sub][s]['restaurants'][rest]['name'],
                                        rating = data[sub][s]['restaurants'][rest]['rating'],
                                        rating_count = data[sub][s]['restaurants'][rest]['rating_count'],
                                        cost = data[sub][s]['restaurants'][rest]['cost'],
                                        address = data[sub][s]['restaurants'][rest]['address'],
                                        cuisine = data[sub][s]['restaurants'][rest]['cuisine'],
                                        lic_no = data[sub][s]['restaurants'][rest]['lic_no'],
                                        location = loc,
                                        link =  data[sub][s]['restaurants'][rest]['link'])
                            restaurant.save()
                        except:
                            try:
                                new = data[sub][s]['restaurants'][rest]['price']
                                restaurant = Restaurant(
                                    rid = rest,
                                    name = data[sub][s]['restaurants'][rest]['name'],
                                    rating = data[sub][s]['restaurants'][rest]['rating'],
                                    cost = data[sub][s]['restaurants'][rest]['price'],
                                    cuisine = data[sub][s]['restaurants'][rest]['type'],
                                    link =  data[sub][s]['restaurants'][rest]['link'],
                                    location = loc,
                                )
                                restaurant.save()
                            except:
                                print(data[sub][s]['restaurants'][rest]['price'])
                                pass
                except:
                    print(data[sub][s])

## Data retriving, cleaning and saving from the database

Data was retrived from the database, cleand and saved in the csv format 

In [2]:
from django.db.models import Count
import pandas as pd


In [ ]:
# all_restaurants = Restaurant.objects.all()
all_restaurants = Restaurant.objects.values('id', 'location__main__name','location__sub__name', 'name', 'rating', 'rating_count', 'cost', ).annotate(total_restaurants=Count('id'))
restaurant_df_new = pd.DataFrame(all_restaurants)
restaurant_df_new.to_csv('clean_swiggy_data.csv', index=False)

In [159]:
# Read the data 
restaurants_df_new = pd.read_csv('clean_swiggy_data.csv')

In [160]:
restaurants_df_new.head(5)

,id,location__main__name,location__sub__name,name,rating,rating_count,cost,total_restaurants
0,1,Abohar,NaN,AB FOODS POINT,--,Too Few Ratings,₹ 200,1
1,2,Abohar,NaN,Janta Sweet House,4.4,50+ ratings,₹ 200,1
2,3,Abohar,NaN,theka coffee desi,3.8,100+ ratings,₹ 100,1
3,4,Abohar,NaN,Singh Hut,3.7,20+ ratings,₹ 250,1
4,5,Abohar,NaN,GRILL MASTERS,--,Too Few Ratings,₹ 250,1


In [165]:
restaurants_df_new['cost'].unique()

array(['₹ 200', '₹ 100', '₹ 250', '₹ 150', '₹ 300', '₹ 700', '₹ 650',
       '₹ 400', '₹ 350', '₹ 60', '₹ 110', '₹ 399', '₹ 249', '₹ 500',
       '₹ 99', '₹ 120', '₹ 499', '₹ 299', '₹ 199', '₹ 50', '₹ 180',
       '₹ 349', '₹ 1000', '₹ 599', '₹ 600', '₹ 800', '₹ 450', '₹ 149',
       '₹ 290', '₹ 175', '₹ 125', '₹ 8', '₹ 375', '₹ 275', '₹ 425',
       '₹ 225', '₹ 325', '₹ 75', '₹ 160', '₹ 550', '₹ 220', '₹ 20',
       '₹ 59', '₹ 1200', '₹ 210', '₹ 30', '₹ 310', '₹ 70', '₹ 170',
       '₹ 449', '₹ 280', nan, '₹ 999', '₹ 3999', '₹ 40', '₹ 235',
       '₹ 1600', '₹ 55', '₹ 179', '₹ 129', '₹ 130', '₹ 240', '₹ 260',
       '₹ 360', '₹ 2', '₹ 140', '₹ 80', '₹ 10', '₹ 85', '₹ 248', '₹ 270',
       '₹ 25', '₹ 850', '₹ 1800', '₹ 230', '₹ 90', '₹ 159', '₹ 370',
       '₹ 1050', '₹ 49', '₹ 12', '₹ 190', '₹ 79', '₹ 340', '₹ 320',
       '₹ 298', '₹ 2000', '₹ 409', '₹ 14', '₹ 198', '₹ 196', '₹ 680',
       '₹ 750', '₹ 888', '₹ 1500', '₹ 1', '₹ 520', '₹ 279', '₹ 900',
       '₹ 1100', '₹ 1400', '₹ 13

In [167]:
# Function to clean cost
dummy_data = {'cost': ['₹ 99 FOR TWO', '₹  99', None, 'tuyt ',  'NA', '0', 0, 7.7, 87]}
df = pd.DataFrame(dummy_data)

# Custom function to process 'cost' values
def process_cost(cost):
    if cost is None or cost == 'NA':
        return 0
    if isinstance(cost, (int, float)):
        return float(cost)
    
    numeric_value = float(pd.Series(cost).str.extract(r'(\d+)')[0].fillna('0'))
    
    if 'FOR' in cost:
        return numeric_value / 2
    return numeric_value

# Apply the custom function using a lambda function
df['costs'] = df['cost'].apply(lambda cost: process_cost(cost))

print(df)



           cost  costs
0  ₹ 99 FOR TWO   49.5
1         ₹  99   99.0
2          None    0.0
3         tuyt     0.0
4            NA    0.0
5             0    0.0
6             0    0.0
7           7.7    7.7
8            87   87.0


In [232]:
restaurants_df_new['rating_count'].unique()

array(['Too Few Ratings', '50+ ratings', '100+ ratings', '20+ ratings',
       '500+ ratings', '1K+ ratings', nan, '5K+ ratings', '10K+ ratings'],
      dtype=object)

In [233]:
# Function to clean cost
dummy_data = {'rating': ['Too Few Ratings', '50+ ratings', '100+ ratings', '20+ ratings',
       '500+ ratings', '1K+ ratings', 'nan', None, '5K+ ratings', '10K+ ratings']}
df_rating = pd.DataFrame(dummy_data)

# Custom function to process 'cost' values
def process_rating_count(rating):
    if rating == 'Too Few Ratings':
        return 5
    if rating == '20+ ratings':
        return 20
    if rating == '50+ ratings':
        return 50
    if rating == '100+ ratings':
        return 100
    if rating == '500+ ratings':
        return 500
    if rating == '1K+ ratings':
        return 1000
    if rating == '5K+ ratings':
        return 5000
    if rating == '10K+ ratings':
        return 10000
    else:
        return 0
    
    numeric_value = float(pd.Series(cost).str.extract(r'(\d+)')[0].fillna('0'))

# Apply the custom function using a lambda function
df_rating['ratings'] = df_rating['rating'].apply(lambda rating: process_rating_count(rating))
df_rating

,rating,ratings
0,Too Few Ratings,5
1,50+ ratings,50
2,100+ ratings,100
3,20+ ratings,20
4,500+ ratings,500
5,1K+ ratings,1000
6,nan,0
7,None,0
8,5K+ ratings,5000
9,10K+ ratings,10000


In [234]:
restaurants_df_new['rating'].unique()

array(['--', '4.4', '3.8', '3.7', '3.6', '4.0', '4.2', '4.7', '4.1',
       '3.5', '3.1', '3.4', '3.3', '4.8', '3.9', '2.7', '4.3', '2.9',
       '4.5', '2.5', '3.2', '2.4', '4.6', '3.0', '2.8', '2.3', '5.0',
       '2.6', '2.2', '1.4', nan, '2.1', '1.3', '2.0', '1.8', '1.6', '4.9',
       '1.9', '1.7', '1.5', '1.2', '1.1', '1.0'], dtype=object)

In [246]:
restaurants_df_new['costs'] = restaurants_df_new['cost'].apply(lambda cost: process_cost(cost))
restaurants_df_new['rating_counts'] = restaurants_df_new['rating_count'].apply(lambda rating: process_rating_count(rating))
restaurants_df_new['ratings'] = restaurants_df_new['rating'].fillna(0).apply(lambda x: float(str(x).replace('--', '0')) if isinstance(x, (str, int, float)) else 0)
restaurants_df = restaurants_df_new.copy()

In [247]:
restaurants_df.drop(['cost', 'rating_count', 'rating'], axis=1, inplace=True)

In [249]:
restaurants_df.head(5)

,id,location__main__name,location__sub__name,name,total_restaurants,costs,rating_counts,ratings
0,1,Abohar,NaN,AB FOODS POINT,1,200.0,5,0.0
1,2,Abohar,NaN,Janta Sweet House,1,200.0,50,4.4
2,3,Abohar,NaN,theka coffee desi,1,100.0,100,3.8
3,4,Abohar,NaN,Singh Hut,1,250.0,20,3.7
4,5,Abohar,NaN,GRILL MASTERS,1,250.0,5,0.0


In [256]:
# rating_count = restaurants_df_new['rating_count'].unique()
# rating_counts = {}
# def cleanRating(rating_count: list) -> dict:
#     for rating in rating_count:
#         if rating == 'Too Few Ratings' or rating == 'NA' or rating is None or rating == 'nan':
#             rating_counts[rating] = 0
#         else:
#             rating_str = str(rating)
#             try:
#                 rating_counts[rating] = int(rating_str.split('+')[0])
#             except:
#                 if rating_str.split('+')[0][-1] == 'K':
#                     rating_counts[rating] = int(rating_str.split('+')[0][:-1]) * 1000
#     return rating_counts
# print(cleanRating(rating_count))

{'Too Few Ratings': 0, '50+ ratings': 50, '100+ ratings': 100, '20+ ratings': 20, '500+ ratings': 500, '1K+ ratings': 1000, '5K+ ratings': 5000, '10K+ ratings': 10000}


# Some primary results were generated during the ETL process

In [43]:
# Number of Cities without subregions
len(set(data.keys()))

623

In [44]:
# Number of Cities have subregions
len(set(location_with_sublocations))

26

In [45]:
# Total number of subregions
len(set(sublocations))

373

In [46]:
# Duplicate names of subregion 
duplicate_sublocations = [x for x in sublocations if sublocations.count(x) > 1]
duplicate_sublocations

['Malviya Nagar', 'Pratap Nagar', 'Malviya Nagar', 'Pratap Nagar']

The set of subregions was less then the original list because there were duplicates

# 1.  How many cities (including subregions) where Swiggy is having its restaurants listed

Total number of cities including the subregions is **996**, **26** cities have subrerions and total subregion is **373**.

In [22]:
totallocation  = len(data.keys()) + len(set(sublocations))
totallocation

996

# 2. How many cities  (don't include subregions) where Swiggy is having their restaurants listed

Total number of main Cities is **623**

In [50]:
len(set(data.keys()))

623

# 3 The Subregion of Delhi with the maximum number of restaurants listed on Swiggy

In Delhi, Indirapuram have the max number of **1279** restaurants.

In [257]:
grouped_df_delhi = restaurants_df[restaurants_df['location__main__name'] == 'Delhi'].groupby(['location__sub__name']).agg(
    number_of_restaurants=('name', 'count')
)
sorted_df_delhi = grouped_df_delhi.sort_values(by='number_of_restaurants', ascending=False)

sorted_df_delhi.head(10)

,number_of_restaurants
location__sub__name,
Indirapuram,1279
Rohini,1137
Greater Kailash 2,1038
Laxmi Nagar,933
Malviya Nagar,900
Dwarka,721
Rajouri Garden,710
GTB Nagar,685
Dilshad Gardens,677


# 4. Name the top 5 Most Expensive Cities in the Datasets.

I have used meidan insted of mean value. Based on the result I could not determine the top 5, so I made a table of top 11. 


In [258]:
grouped_df_mec = restaurants_df.groupby(['location__main__name']).agg(
    Costs=('costs', 'median')
)

sorted_df_mec = grouped_df_mec.sort_values(by='Costs', ascending=False)
sorted_df_mec.head(11)

,Costs
location__main__name,
Gangtok,500.0
Mussoorie,450.0
Diu,400.0
Darjeeling,400.0
North-goa,400.0
Budhwal,400.0
Central-goa,400.0
Rangpo,400.0
Kohima,400.0


# 5. List out the top 5 Restaurants with Maximum & minimum ratings throughout the dataset


I have used mean for rating and sum for the total number of ratings count. The avg rating may not reresent the correct evaluation, as, a restaurant may have many outlates and number of rating count also could be heigher than the a restauent have 1 outlate with few visitor. From the below tables we can see that most popular restaurants with many outlates have very poor avg ratings.

In [269]:
grouped_dataset = restaurants_df.groupby(['name']).agg(costs =('costs', 'mean'), avg_ratings = ('ratings', 'mean'), num_ratings = ('rating_counts','sum'), outlates = ('name','count'))

In [270]:
grouped_dataset.sort_values(by='avg_ratings', ascending=False).head(5)


,costs,avg_ratings,num_ratings,outlates
name,,,,
MY KEBAB BITE,600.0,5.0,40,2
Mr. M G FOOD,200.0,5.0,20,1
OLIVE PIZZERIA,200.0,5.0,20,1
naushera,500.0,5.0,20,1
Not Just A Cake by Jayaa Kennethh,400.0,5.0,20,1


In [271]:
grouped_dataset.sort_values(by='avg_ratings', ascending=False).tail(5)

,costs,avg_ratings,num_ratings,outlates
name,,,,
JAGADAMBA MITHAI BHANDAR,300.0,0.0,5,1
JAGADAMB VEJ CHINESE,150.0,0.0,5,1
JAGA FOOD COURT,300.0,0.0,5,1
JAGA DAHIBARA,100.0,0.0,5,1
zzz pizza,400.0,0.0,5,1


In [272]:
grouped_dataset.sort_values(by='num_ratings', ascending=False).head(5)


,costs,avg_ratings,num_ratings,outlates
name,,,,
McDonald's,395.073892,4.184729,133950,203
KFC,404.875000,3.903659,122055,328
Burger King,340.217391,4.062112,118960,161
NIC Natural Ice Creams,118.421053,4.487368,72075,190
Mehfil,300.000000,2.785714,71120,14


In [273]:
grouped_dataset.sort_values(by='num_ratings', ascending=False).tail(5)

,costs,avg_ratings,num_ratings,outlates
name,,,,
Pretty Baked,75.0,0.0,0,1
Pritis Rannaghar,125.0,4.3,0,1
Priyadarshini Hotel,150.0,3.8,0,1
Priyanshu Pavitra Bhojnalaya,50.0,0.0,0,1
MAHAVEER PRIME RABDI BHANDAR AND BHOJNALAY,150.0,0.0,0,1


# 6. Name of top 5 cities with the highest number of restaurants listed.


In [274]:
# Name of top 10 cities with the highest number of restaurants listed.

grouped_df_total = restaurants_df.groupby(['location__main__name']).agg(
    number_of_restaurants=('name', 'count')
)

sorted_df_total = grouped_df_total.sort_values(by='number_of_restaurants', ascending=False)
sorted_df_total.head(10)


,number_of_restaurants
location__main__name,
Bangalore,15302
Delhi,14080
Pune,12440
Hyderabad,11888
Chennai,9954
Kolkata,8282
Mumbai,6103
Jaipur,5540
Gurgaon,3599


# 7. Top 10 cities as per the number of restaurants listed

I have listed the top 10 cities in Q 6. So, here I have listed the top subregions with total restautant number

In [275]:
# Top 10 cities as per the number of restaurants listed

grouped_df_top = restaurants_df.groupby(['location__main__name','location__sub__name'])['total_restaurants'].sum().reset_index()

# Sort in descending order
grouped_df_top = grouped_df_top.sort_values(by='total_restaurants', ascending=False)

grouped_df_top.head(10)

,location__main__name,location__sub__name,total_restaurants
113,Delhi,Indirapuram,1279
13,Bangalore,BTM,1172
126,Delhi,Rohini,1138
279,Pune,Kothrud,1089
23,Bangalore,Indiranagar,1086
140,Gurgaon,sohna road,1081
19,Bangalore,Electronic City,1049
110,Delhi,Greater Kailash 2,1038
252,Mumbai,Vashi,1022
160,Hyderabad,Kukatpally,1009


# 8. Name the top 5 Most Popular Restaurants in Pune

In [276]:
# Name the top 5 Most Popular Restaurants in Pune
grouped_df_pune = restaurants_df[restaurants_df['location__main__name'] == 'Pune'].groupby(['name'])\
    .agg({'costs': 'mean', 'ratings': 'mean', 'rating_counts':'sum', 'name':'count'})

In [277]:
# Name the top 5 Most Popular Restaurants in Pune based on number of ratings
grouped_df_pune.sort_values(by='rating_counts', ascending=False).head(5)

,costs,ratings,rating_counts,name
name,,,,
McDonald's,400.000000,4.225000,15200,20
Burger King,350.000000,4.040000,9500,10
KFC,411.538462,3.823077,5920,13
NIC Natural Ice Creams,120.000000,4.600000,5100,15
Irani Cafe,241.666667,4.350000,5000,6


In [278]:
# Name the top 5 Most Popular Restaurants in Pune based on avg ratings

grouped_df_pune.sort_values(by='ratings', ascending=False).head(5)

,costs,ratings,rating_counts,name
name,,,,
The Samosa Man,250.0,5.0,20,1
Roll Over Biryani,250.0,5.0,20,1
Miss Gelato,200.0,5.0,20,1
Kushta by Semporo,800.0,5.0,20,1
Northie,250.0,5.0,20,1


# 9. Which SubRegion in Delhi is having the least expensive restaurant in terms of cost

In [279]:
# SubRegions in Delhi having the most and least expensive restaurant in terms of cost

grouped_df_delhi2 = restaurants_df[restaurants_df['location__main__name'] == 'Delhi'].groupby(['location__main__name', 'location__sub__name']).agg(
    {'costs': 'mean', 'ratings': 'mean', 'rating_counts':'mean'}
)
sorted_df_delhi2 = grouped_df_delhi2.sort_values(by='costs', ascending=False)

# SubRegions in Delhi having the most expensive restaurant
print (sorted_df_delhi2.head(5))

# SubRegions in Delhi having the least expensive restaurant
print (sorted_df_delhi2.tail(5))


                                               costs   ratings  rating_counts
location__main__name location__sub__name                                     
Delhi                BBK_Chattarpur       800.000000  0.000000       5.000000
                     Khan Market          600.915888  1.631776      26.682243
                     Vasant Kunj          439.375000  1.605263      44.177632
                     Connaught Place      434.449612  1.946512      69.922481
                     Greater Kailash New  406.172840  2.781481     119.197531
                                               costs   ratings  rating_counts
location__main__name location__sub__name                                     
Delhi                Rohini               277.913808  1.036028      38.633568
                     Mayur Vihar          275.059259  1.082074      34.859259
                     Laxmi Nagar          270.195070  1.026902      41.998928
                     Uttam Nagar          253.321970  0.731439  

# 10. Top 5 most popular restaurant chains in India

In [280]:
# Top 5 most popular restaurant chains in India based on rating count

grouped_df_all = restaurants_df.groupby('name')['rating_counts'].sum().reset_index()

# Sort the grouped DataFrame by 'total_restaurants' in descending order
grouped_df_all = grouped_df_all.sort_values(by='rating_counts', ascending=False)

# sorted_df.head(5)
grouped_df_all.set_index('name').head(5)

,rating_counts
name,
McDonald's,133950
KFC,122055
Burger King,118960
NIC Natural Ice Creams,72075
Mehfil,71120


# 11.  Which restaurant in Pune has the most number of people visiting

**NH1 Bowls - Highway To North** is the restaurant where most number of people are visiting at Pune.

In [281]:
# Which restaurant in Pune has the most number of people visiting based on number of ratings.
grouped_df_pune.sort_values(by='rating_counts', ascending=False).head(5)

,costs,ratings,rating_counts,name
name,,,,
McDonald's,400.000000,4.225000,15200,20
Burger King,350.000000,4.040000,9500,10
KFC,411.538462,3.823077,5920,13
NIC Natural Ice Creams,120.000000,4.600000,5100,15
Irani Cafe,241.666667,4.350000,5000,6


# 12. Top 10 Restaurants with Maximum Ratings in Banglore

I have creatd two table, one based on ave ratings and another number of ratings.

In [283]:
# Top 10 Restaurants with Maximum Ratings in Banglore

grouped_df_banglore = restaurants_df[restaurants_df['location__main__name'] == 'Bangalore'].groupby(['name']).agg({'costs': 'mean', 'ratings': 'mean', 'rating_counts':'mean', 'name':'count'})

sorted_df_ratings_banglore = grouped_df_banglore.sort_values(by='ratings', ascending=False)
sorted_df_rating_counts_banglore = grouped_df_banglore.sort_values(by='rating_counts', ascending=False)

print(sorted_df_ratings_banglore.head(10))

print(sorted_df_rating_counts_banglore.head(10))

                                   costs  ratings  rating_counts  name
name                                                                  
Shawarma Indiah                    325.0      5.0           20.0     2
Oh Bangla                          250.0      5.0           20.0     1
Minus 30                           250.0      5.0           20.0     1
Fabelle Chocolates - ITC Gardenia  600.0      5.0           20.0     1
Fresh crunch pizza                 400.0      5.0           20.0     1
Joao's Croissant Fusion            350.0      5.0           20.0     1
MOKA ON THE GO                     300.0      5.0           20.0     1
Mels SubHub                        350.0      5.0           20.0     1
Bansal's Thali & Combo Meals       300.0      5.0           20.0     1
PEDRO DE GOA                       350.0      5.0           20.0     1
                           costs   ratings  rating_counts  name
name                                                           
Truffles            

# 13. Top 10 Restaurant in Patna w.r.t rating

I could not find top 10 based on rating or rating_count, because 10 restaurant have same avg ratings and rating_count.

In [284]:
grouped_df_patna = restaurants_df[restaurants_df['location__main__name'] == 'Patna'].groupby(['name'])\
    .agg(costs =('costs', 'mean'), avg_ratings = ('ratings', 'mean'), num_ratings = ('rating_counts','sum'), outlates = ('name','count'))


In [285]:
grouped_df_patna.sort_values(by='avg_ratings', ascending=False).head(50)

,costs,avg_ratings,num_ratings,outlates
name,,,,
Kwality Walls Frozen Dessert and Ice Cream Shop,100.0,4.6,1000,1
Slurpy Shakes,150.0,4.6,50,1
Burger It Up,200.0,4.5,20,1
GARAGE KITCHEN,300.0,4.5,50,1
Kouzina Kafe The Food Court,200.0,4.5,50,1
Cupcake Bliss Cake & Desserts,200.0,4.5,20,1
Indiana Burgers,200.0,4.4,50,1
KINGDOM OF MOMOS,400.0,4.4,50,1
Pure Veg Meals by Lunchbox,200.0,4.4,20,1


In [286]:
grouped_df_patna.sort_values(by='num_ratings', ascending=False).head(50)

,costs,avg_ratings,num_ratings,outlates
name,,,,
Sujata Family Restaurant,300.0,3.4,5000,1
Hungry Hope,300.0,3.5,5000,1
0612 ROAD CHEF [ Open Air ],200.0,3.9,1000,1
Khana Khazana,300.0,3.4,1000,1
Brijwashi Sweets And Restaurant,150.0,3.9,1000,1
Cake Palace,250.0,3.6,1000,1
Chandan Dhaba,150.0,3.4,1000,1
Classic Family Restaurant,400.0,3.1,1000,1
Dosa Station,150.0,3.0,1000,1
